# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [17]:
%load_ext dotenv
%dotenv 
%reload_ext dotenv



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


In [18]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [19]:
import os
from glob import glob
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
from dotenv import load_dotenv
import pandas as pd

# Load the environment variables from the .env file
load_dotenv()

# Access the PRICE_DATA environment variable
price_data_path = os.getenv('PRICE_DATA')


# Use glob to find the path of all parquet files in the directory PRICE_DATA, including nested subdirectories
parquet_files = glob(os.path.join(price_data_path, '**', '*.parquet'), recursive=True)


# Filter out any directories from the list of parquet files
parquet_files = [file for file in parquet_files if os.path.isfile(file)]


# Load all the parquet files into a Dask DataFrame
dask_df = dd.read_parquet(parquet_files, engine='pyarrow')

# Display the first few rows and the columns of the DataFrame to verify the structure and data
print(dask_df.head())
print("Columns in the DataFrame:", dask_df.columns)


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.532219   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.206844   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.712799   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.276852   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.299923   

         Volume       sector                       subsector  year  
ticker                                                              
A       4674353  Health Care  Life Sciences Tools & Services  2000  
A       4765083  Health Care  Life Sciences Tools & Services  2000  
A       5758642  Health Care  Life Sciences Tools & Services  2000  
A       2534434  Health Care  Life Sciences Tools & Services  2000  
A       2819626  Health Care  Life Sciences Tools & S

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [20]:
# Load all the parquet files into a Dask DataFrame
dask_df = dd.read_parquet(parquet_files, engine='pyarrow')

# Define the function to add features
def add_features(df):
    df = df.sort_index()
    df['Close_lag'] = df['Close'].shift(1)
    df['Adj_Close_lag'] = df['Adj Close'].shift(1)
    df['returns'] = (df['Adj Close'] / df['Adj_Close_lag']) - 1
    df['hi_lo_range'] = df['High'] - df['Low']
    return df

# Create a sample DataFrame to define metadata
sample_df = pd.DataFrame({
    'Date': pd.to_datetime([]),
    'Open': pd.Series([], dtype='float64'),
    'High': pd.Series([], dtype='float64'),
    'Low': pd.Series([], dtype='float64'),
    'Close': pd.Series([], dtype='float64'),
    'Adj Close': pd.Series([], dtype='float64'),
    'Volume': pd.Series([], dtype='int64'),
    'sector': pd.Series([], dtype='object'),
    'subsector': pd.Series([], dtype='object'),
    'year': pd.Series([], dtype='int64'),
    'Close_lag': pd.Series([], dtype='float64'),
    'Adj_Close_lag': pd.Series([], dtype='float64'),
    'returns': pd.Series([], dtype='float64'),
    'hi_lo_range': pd.Series([], dtype='float64'),
})

# Apply the feature addition function using map_partitions with accurate metadata
dd_feat = dask_df.map_partitions(lambda df: df.groupby('ticker').apply(add_features), meta=sample_df)




+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [21]:
# Display the first few rows to verify the features were added correctly
print(dd_feat.head())

# Convert the Dask DataFrame to a pandas DataFrame
pandas_df = dd_feat.compute()

# Add a rolling average return calculation with a window of 10 days
pandas_df['rolling_avg_return'] = pandas_df['returns'].rolling(10).mean()

# Display the first few rows to verify the rolling average return was added correctly
print(pandas_df.head())



                    Date       Open       High        Low      Close  \
ticker ticker                                                          
A      A      2000-01-03  56.330471  56.464592  48.193848  51.502148   
       A      2000-01-04  48.730328  49.266811  46.316166  47.567955   
       A      2000-01-05  47.389126  47.567955  43.141991  44.617310   
       A      2000-01-06  44.080830  44.349072  41.577251  42.918453   
       A      2000-01-07  42.247852  47.165592  42.203148  46.494991   

               Adj Close   Volume       sector  \
ticker ticker                                    
A      A       43.532219  4674353  Health Care   
       A       40.206844  4765083  Health Care   
       A       37.712799  5758642  Health Care   
       A       36.276852  2534434  Health Care   
       A       39.299923  2819626  Health Care   

                                    subsector  year  Close_lag  Adj_Close_lag  \
ticker ticker                                                  

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.